# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,suklayin,15.7574,121.5503,296.90,94,31,1.07,PH,1702146163
1,1,whitehorse,60.7161,-135.0538,261.58,84,100,1.54,CA,1702146163
2,2,port-aux-francais,-49.3500,70.2167,276.61,79,65,11.61,TF,1702146164
3,3,jamestown,42.0970,-79.2353,287.35,59,100,6.69,US,1702146164
4,4,waitangi,-43.9535,-176.5597,288.05,86,82,6.26,NZ,1702146164


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x="Lng",
    y="Lat",
    c="Humidity",
    colormap="viridis",
    size="Humidity",
    title="City Humidity Map",
    xlabel="Longitude",
    ylabel="Latitude",
    clim=(0, city_data_df["Humidity"].max()),  # Set color limit based on maximum humidity
    colorbar=True,
    frame_width=500,
    frame_height=500
)

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 293.15) & (city_data_df["Max Temp"] < 298.15) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
156,156,colotlan,22.1000,-103.2667,295.67,34,0,3.07,MX,1702146179
243,243,alaghsas,17.0187,8.0168,298.09,14,0,3.60,NE,1702146195
558,558,gilmer,32.7287,-94.9424,297.13,49,0,1.54,US,1702146245


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Create hotel_df with selected columns
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
156,colotlan,MX,22.1000,-103.2667,34,
243,alaghsas,NE,17.0187,8.0168,14,
558,gilmer,US,32.7287,-94.9424,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
from api_keys import geoapify_key

import requests
from requests.structures import CaseInsensitiveDict

url = "https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=rect%3A10.716463143326969%2C48.755151258420966%2C10.835314015356737%2C48.680903341613316&limit=20&apiKey=edc25c8a398b401799c61aba7793f8c3"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)

# Set parameters for the Geoapify Places API
geoapify_url = "https://api.geoapify.com/v2/places"
geoapify_api_key = geoapify_key


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Set parameters for the Geoapify Places API request
    params = {
        "categories": "tourism.hotels",
        "radius": 100000,  # Adjust the radius as needed
        "limit": 1,  # Limit to 1 result per request
        "apiKey": geoapify_api_key
    }

    # Add latitude and longitude to the parameters
    params["bias"] = f"circle:{params['radius']}@{lng},{lat}"

    # Set up headers for the request
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"

    # Make the Geoapify Places API request
    response = requests.get(geoapify_url, params=params, headers=headers)
    result = response.json()

    # Make the Geoapify Places API request
    response = requests.get(geoapify_url, params=params, headers=headers)
    result = response.json()

    # Print the API response for debugging
    print(result)

    # Extract hotel information and update hotel_df
    try:
        hotel_name = result["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the updated hotel_df
print(hotel_df)

200
{'statusCode': 401, 'error': 'Unauthorized', 'message': 'Invalid apiKey'}
colotlan - nearest hotel: No hotel found
{'statusCode': 401, 'error': 'Unauthorized', 'message': 'Invalid apiKey'}
alaghsas - nearest hotel: No hotel found
{'statusCode': 401, 'error': 'Unauthorized', 'message': 'Invalid apiKey'}
gilmer - nearest hotel: No hotel found
         City Country      Lat       Lng  Humidity      Hotel Name
156  colotlan      MX  22.1000 -103.2667        34  No hotel found
243  alaghsas      NE  17.0187    8.0168        14  No hotel found
558    gilmer      US  32.7287  -94.9424        49  No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

hotel_map = hotel_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    size='Humidity',
    hover_cols=['City', 'Country', 'Hotel Name'],
    title='Hotel Information by City',
    xlabel='Longitude',
    ylabel='Latitude',
    cmap='viridis',
    colorbar=True,
    height=600,
    width=800,
)

# Show the map
hotel_map

:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)